In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics

In [2]:
using Pkg
Pkg.add("PyCall")

    Updating registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
n_stations_test=20

20

In [4]:
# Read the data
stations_info = CSV.read("../data/stations/station_information.csv", Tables.matrix, header=1);
#stations_info

449×15 Matrix{Any}:
   3  "classic"  …  "B32006"  42.3401   10
   4  "classic"     "C32000"  42.3454   10
   5  "classic"     "B32012"  42.3418   10
   6  "classic"     "D32000"  42.3613   10
   7  "classic"     "A32000"  42.3534   10
   8  "classic"  …  "A32001"  42.3533   10
   9  "classic"     "A32002"  42.3517   10
  10  "classic"     "A32003"  42.3504   10
  11  "classic"     "A32004"  42.3386   10
  12  "classic"     "B32002"  42.3362   10
   ⋮             ⋱                     
 575  "classic"  …  "T32014"  42.5209  136
 576  "classic"     "F32001"  42.4117  139
 577  "classic"     "F32002"  42.4177  139
 578  "classic"     "F32003"  42.4017  139
 582  "classic"     "M32081"  42.3731    8
 583  "classic"  …  "M32082"  42.363     8
 584  "classic"     "G32001"  42.4265  140
 585  "classic"     "G32002"  42.4239  140
 586  "classic"     "V32015"  42.4082  104

In [5]:
n_stations=length(stations_info[:,1])

449

## Let's build our input data matrices

In [6]:
# Capacities C_i
C=stations_info[:,9][:,:];
C_test=C[1:n_stations_test,:];

In [7]:
# Distances D_ij
D=Array{Float64}(undef,n_stations,n_stations)
for i=1:n_stations
    for j=1:n_stations
        D[i,j]=sqrt((stations_info[i,6]-stations_info[j,6])^2+(stations_info[i,14]-stations_info[j,14])^2)
    end
end
D_test=D[1:n_stations_test,1:n_stations_test]

20×20 Matrix{Float64}:
 0.0         0.0314487   0.0105772   …  0.0209635   0.0466574   0.0469228
 0.0314487   0.0         0.020872       0.0110992   0.0156188   0.0180747
 0.0105772   0.020872    0.0            0.0106208   0.0361487   0.0367712
 0.041174    0.0164451   0.0315853      0.0212939   0.0133178   0.00800714
 0.0575982   0.0262908   0.0470539      0.0367346   0.0110427   0.0141801
 0.0390026   0.0681613   0.0485214   …  0.0570635   0.0817742   0.0798796
 0.0217526   0.0498188   0.0304998      0.0387241   0.0634897   0.0618298
 0.0128289   0.0389868   0.0200358      0.0278932   0.0527617   0.0513606
 0.00606602  0.0374989   0.0166289      0.0270202   0.0527229   0.052916
 0.0132129   0.0205212   0.00598588     0.0126208   0.0361393   0.0378907
 0.0278012   0.00963231  0.0179482   …  0.0126511   0.0235761   0.0273043
 0.00350789  0.0341882   0.0134226      0.0239968   0.0495622   0.050083
 0.0428823   0.0700345   0.051512       0.0590481   0.0827474   0.0801761
 0.0253315   0.0

In [8]:
# Feasibility X_ij
threshold=0.01
X=Array{Float64}(undef,n_stations,n_stations)
for i=1:n_stations
    for j=1:n_stations
        if D[i,j]<threshold
            X[i,j]=1
        else
            X[i,j]=0
        end
    end
end
#X
X_test=X[1:n_stations_test,1:n_stations_test]

20×20 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  

In [9]:
# Let's count the non-zero values in X
sum(X[i,12] > 0 for i in 1:n_stations)

13

In [10]:
# Demand d_ijt
using PyCall

py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

d = load_pickle("../data/parameters/demand_matrix.pkl");

In [11]:
d_test=d[1:n_stations_test,1:n_stations_test,:]

20×20×744 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.

In [12]:
sum(d_test[:,1,:])

207.0

In [13]:
# Number of vans
K=2
# Capacity of vans
S=10

10

In [16]:
# Initial number of bikes available y0 at midnight
y0=Array{Int64}(undef,n_stations)
for i in 1:n_stations
    y0[i]=floor(C[i]/2)
end
y0_test=y0[1:n_stations_test];

BoundsError: BoundsError: attempt to access 4-element Vector{Int64} at index [1:20]

In [17]:
# capacity of stations
C = [10,10,10,10];

# demand of stations
d = [[[0,10,0,5] [0,0,0,0] [0,5,0,0] [20,0,0,0]];;;
    [[0,0,0,0] [0,0,0,0] [30,0,0,0] [0,10,0,0]];;;
    [[0,0,0,5] [0,0,15,0] [0,0,0,0] [0,0,0,0]]];

# proximity of stations
X = [[0,1,1,1] [1,0,1,1] [1,1,0,1] [1,1,1,0]];

# capacity of stations
y0 = [6,2,4,3];

# number of vans
K = 2;

# capacity of vans
S = 20;

# distance of stations
D = [[0,0.5,1,0.5] [2,0,0.5,1] [0.5,1,0,0.5] [1,2,1,0]];

# number of stations
n_stations = 4;
n_hours=3;

## Simplified formulation

In [30]:
function first_model(C,D,X,y0,K,S,d,n_stations,n_hours)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 100)
    M=30
    lambda=0.01 # to be tuned
    n_rebal_stations = 4
    # Decision variables
    @variable(model, x[1:n_rebal_stations, 1:n_rebal_stations, 1:K, 1:n_hours], Bin)
    @variable(model, 0 <= z[1:n_rebal_stations, 1:n_rebal_stations, 1:K, 1:n_hours], Int)
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints:
    # Stations capacity
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    # No rebalancing at the same station
    @constraint(model, [i in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], x[i,i,k,t] == 0)
    # Flow balance 
    @constraint(model, [i in 1:n_rebal_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)-sum(z[i,j,k,t] for j in 1:n_rebal_stations, k in 1:K)+sum(z[j,i,k,t] for j in 1:n_rebal_stations, k in 1:K))
    @constraint(model, [i in n_rebal_stations+1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations))
    # Flow balance for the first hour 
    @constraint(model, [i in 1:n_rebal_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations)-sum(z[i,j,k,1] for j in 1:n_rebal_stations, k in 1:K)+sum(z[j,i,k,1] for j in 1:n_rebal_stations, k in 1:K))
    @constraint(model, [i in n_rebal_stations+1:n_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations))
    # Vans capacity
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S)
    # Users travel availability
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    # Max 1 rebalancing per hour per van
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(x[i,j,k,t] for i in 1:n_rebal_stations, j in 1:n_rebal_stations) <= 1)
    # Rebalancing feasibility
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= M*x[i,j,k,t])
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], x[i,j,k,t] <= X[i,j])
    # Users travel is inferior to the demand and definition of u
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], d[i,j,t] >= w[i,j,t])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)+lambda*sum(D[i,j]*x[i,j,k,t] for i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

first_model (generic function with 1 method)

In [31]:
x,w,u,y,z,obj=first_model(C,D,X,y0,K,S,d,n_stations,n_hours)
#x, w, u, obj=first_model(C_test,D_test,X_test,y0_test,K,S,d_test,n_stations_test)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Set parameter TimeLimit to value 100
Objective value: 40.05


([0.0 -0.0 -0.0 -0.0; -0.0 0.0 -0.0 -0.0; -0.0 1.0 0.0 -0.0; -0.0 -0.0 -0.0 0.0;;; 0.0 -0.0 -0.0 -0.0; -0.0 0.0 -0.0 -0.0; -0.0 -0.0 0.0 -0.0; -0.0 -0.0 1.0 0.0;;;; 0.0 -0.0 -0.0 -0.0; -0.0 0.0 -0.0 -0.0; 1.0 -0.0 0.0 0.0; -0.0 -0.0 -0.0 0.0;;; 0.0 -0.0 -0.0 -0.0; -0.0 0.0 -0.0 -0.0; -0.0 -0.0 0.0 -0.0; -0.0 1.0 -0.0 0.0;;;; 0.0 -0.0 0.0 -0.0; 0.0 0.0 1.0 -0.0; -0.0 -0.0 0.0 -0.0; -0.0 -0.0 -0.0 0.0;;; 0.0 -0.0 -0.0 1.0; -0.0 0.0 0.0 -0.0; -0.0 -0.0 0.0 -0.0; -0.0 -0.0 -0.0 0.0], [0.0 0.0 0.0 15.0; 9.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0; 5.0 0.0 0.0 0.0;;; 0.0 0.0 5.0 0.0; 0.0 0.0 0.0 10.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 10.0 0.0 0.0; 5.0 0.0 0.0 0.0], [0.0 0.0 0.0 5.0; 1.0 0.0 4.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;;; 0.0 0.0 25.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 5.0 0.0 0.0; 0.0 0.0 0.0 0.0], [5.0 5.0 0.0; 10.0 10.0 0.0; 0.0 0.0 10.0; 0.0 0.0 5.0], [-0.0 0.0 0.0 -0.0; -0.0 -0.0 -0.0

In [35]:
z[:,:,1,2]

4×4 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0
  0.0  -0.0  -0.0  -0.0
  5.0   0.0  -0.0   0.0
  0.0  -0.0  -0.0  -0.0